# Differential Cell-cell communication with scSeqComm-CrossTalkeR

This notebook outlines the steps of inference, analysis and visualization of differential cell-cell communication for a single-cell RNA sequencing data using scSeqComm-CrossTalkeR.

The source code for CrossTalkeR is available on GitHub: [CostaLab/CrossTalkeR](https://github.com/CostaLab/CrossTalkeR) 

The source code for scSeqComm is available on Gitlab: [sysbiobig/scseqcomm](https://gitlab.com/sysbiobig/scseqcomm)

In [1]:
library(Seurat)
library(CrossTalkeR)
library(scSeqComm)
library(tidyverse)
library(data.table)
library(EnhancedVolcano)
require(ggplot2)

Loading required package: SeuratObject

Loading required package: sp

Loading required package: sp


Attaching package: 'SeuratObject'


The following objects are masked from 'package:base':

    intersect, t




Warning message:
"replacing previous import 'dplyr::as_data_frame' by 'igraph::as_data_frame' when loading 'CrossTalkeR'"
Warning message:
"replacing previous import 'dplyr::groups' by 'igraph::groups' when loading 'CrossTalkeR'"
Warning message:
"replacing previous import 'clusterProfiler::simplify' by 'igraph::simplify' when loading 'CrossTalkeR'"
Warning message:
"replacing previous import 'dplyr::union' by 'igraph::union' when loading 'CrossTalkeR'"




Warning message:
"replacing previous import 'igraph::as_data_frame' by 'tibble::as_data_frame' when loading 'CrossTalkeR'"
Warning message:
"replacing previous import 'assertthat::has_name' by 'tibble::has_name' when loading 'CrossTalkeR'"
Warning message:
"replacing previous import 'igraph::crossing' by 'tidyr::crossing' w

### Looking into the scRNAseq data

In the current tutorial, we will utilize the Myocardial Infarction data from [Kuppe 2022](https://www.nature.com/articles/s41586-022-05060-x), which comprises heart samples from 23 patients. From each sample, scRNAseq and SRT were obtained. For the interest of time, we focus our tutorial in studying only the Ishemic and Myogenic phenotypes

In [2]:
data <- readRDS("/Tutorial_ISMBECCB2025/data_vt3/second/crosstalker/2306_scRNAseq_MOIZ.rds")

Warning message in gzfile(file, "rb"):
"cannot open compressed file '/Tutorial_ISMBECCB2025/data_vt3/second/crosstalker/2306_scRNAseq_MOIZ.rds', probable reason 'No such file or directory'"


ERROR: Error in gzfile(file, "rb"): cannot open the connection


In [ ]:
table(data$patient_group)

## scSeqComm for each phenotype - only intercellular communication

In [ ]:
myogenicmtx <- subset(data, patient_group=="myogenic") ## Subsetting the data for only myogenic cells
myogenicmeta <- data.frame(Cell_ID=colnames(myogenicmtx), Cluster_ID=myogenicmtx$cell_subtype2)

### Now we call scSeqComm function to perform the intercellular inference in the phenotypes we want to compare

In [ ]:
data(LR_pairs_ConnectomeDB_2020) ## Selecting the LR resource
LR_db <- LR_pairs_ConnectomeDB_2020

In [ ]:
num_core <- 8
res_myo <- scSeqComm_analyze(gene_expr = GetAssayData(myogenicmtx,'RNA'),
                                  cell_metadata = myogenicmeta,
                                  LR_pairs_DB = LR_db,
                                  inter_signaling = T,                      # wheter comppute intercellular signaling
                                  inter_scores = "scSeqComm",               # Intercellular signaling scoring schemes to be computed (default "scSeqComm")
                                  min_cells = 30,                           # Minimum number of cells that a cluster (i.e., cell type) should be composed of to compute scSeqComm score
                                  intra_signaling = F                      # whether compute intracellular signaling

                                  )

In [ ]:
rm(myogenicmtx) 
gc()

### In analogous manner, we will infer the cell cell communication using the Ischemic samples

In [ ]:
ischemicmtx <- subset(data, patient_group=="ischemic") ## Subsetting the data for only ischemic cells
ischemicmeta <- data.frame(Cell_ID=colnames(ischemicmtx), Cluster_ID=ischemicmtx$cell_subtype2)

In [ ]:
res_isch <- scSeqComm_analyze(gene_expr = GetAssayData(ischemicmtx,'RNA'),
                                  cell_metadata = ischemicmeta,
                                  LR_pairs_DB = LR_db,
                                  inter_signaling = T,                      # wheter comppute intercellular signaling
                                  inter_scores = "scSeqComm",               # Intercellular signaling scoring schemes to be computed (default "scSeqComm")
                                  min_cells = 30,                           # Minimum number of cells that a cluster (i.e., cell type) should be composed of to compute scSeqComm score
                                  intra_signaling = F                      # whether compute intracellular signaling

                                  )

In [ ]:
rm(ischemicmtx) 
rm(data)
gc()

### Following, we use a threshold to filter the relevant interactions

In [ ]:
selected_comm_myo <- scSeqComm_select(res_myo$comm_results, 
                                  S_inter = 0.5,
                                  operator = "OR")
selected_comm_isc <- scSeqComm_select(res_isch$comm_results, 
                                  S_inter = 0.5,
                                  operator = "OR")

### Preparing the data for CrossTalkeR

In [ ]:
selected_comm_myo<-selected_comm_myo %>%
    mutate(source=cluster_L) %>%
    mutate(target=cluster_R) %>%
    mutate(gene_A=ligand) %>%
    mutate(gene_B=receptor) %>%
    mutate(type_gene_A="Ligand") %>%
    mutate(type_gene_B="Receptor") %>%
    mutate(LRScore=round(S_inter,4)) %>%
    select(c("source", "target", "gene_A", "gene_B", "type_gene_A", "type_gene_B","LRScore"))%>%
    filter(!is.na(.data$LRScore)) %>% filter(.data$LRScore > 0.2)
selected_comm_isc<-selected_comm_isc %>%
    mutate(source=cluster_L) %>%
    mutate(target=cluster_R) %>%
    mutate(gene_A=ligand) %>%
    mutate(gene_B=receptor) %>%
    mutate(type_gene_A="Ligand") %>%
    mutate(type_gene_B="Receptor") %>%
    mutate(LRScore=round(S_inter,4))%>%
    select(c("source", "target", "gene_A", "gene_B", "type_gene_A", "type_gene_B","LRScore")) %>%
    filter(!is.na(.data$LRScore)) %>%
    filter(.data$LRScore > 0.2)

## CrossTalkeR

In [ ]:
paths <- list("myogenic"=selected_comm_myo,
             "ischemic"=selected_comm_isc)

ctkerdata<-generate_report(lrpaths = paths,
                threshold = 0,
                out_path = "/Tutorial_ISMBECCB2025/data_vt3/second/crosstalker/",
                out_file = "out.html",
                sel_columns = c("source", "target", "gene_A", "gene_B", "type_gene_A", "type_gene_B","LRScore"),filtered=TRUE)

In [ ]:
dir("/Tutorial_ISMBECCB2025/data_vt3/second/crosstalker/")

### Looking into the differential abundant CCC pairs

In [ ]:
options(repr.plot.width=15,repr.plot.height=10)
EnhancedVolcano(ctkerdata@stats$ischemic_x_myogenic,
                lab = ctkerdata@stats$ischemic_x_myogenic$columns_name,
                x = "lodds",
                y = "p",
                pCutoff = 0.05,FCcutoff=0.5)

### Now we can use the filtered version to check which cells are communicating more and better

In [ ]:
options(repr.plot.width=21,repr.plot.height=7)
p1<-ctkerdata@rankings$ischemic_x_myogenic_filtered |>
    ggplot(aes(x=Influencer,y=reorder(nodes,Influencer),fill=Influencer))+ 
          geom_bar(stat = "identity")+
          scale_fill_viridis_c()+
          theme_minimal()+
          theme(axis.text.y = element_text(color = "grey20", size = 18, angle = 0, hjust = 1, vjust = 0, face = "plain"))
p2<-ctkerdata@rankings$ischemic_x_myogenic_filtered |>
    ggplot(aes(x=Listener,y=reorder(nodes,Listener),fill=Listener))+ 
          geom_bar(stat = "identity")+
          scale_fill_viridis_c()+
          theme_minimal()+
          theme(axis.text.y = element_text(color = "grey20", size = 18, angle = 0, hjust = 1, vjust = 0, face = "plain"))
p3<-ctkerdata@rankings$ischemic_x_myogenic_filtered |>
    ggplot(aes(x=Pagerank,y=reorder(nodes,Pagerank),fill=Pagerank))+ 
          geom_bar(stat = "identity")+
          scale_fill_viridis_c()+
          theme_minimal()+
          theme(axis.text.y = element_text(color = "grey20", size = 18, angle = 0, hjust = 1, vjust = 0, face = "plain"))
p1+p2+p3

In [ ]:
ctkerdata@tables$ischemic_x_myogenic %>%
    slice_max(order_by = -abs(LRScore),with_ties = FALSE,n=10)

In [ ]:
options(repr.plot.width=20,repr.plot.height=10)
threshold = 25
data <- ctkerdata@tables$ischemic_x_myogenic %>% 
        filter(source=="damaged_CM") %>%
        filter(gene_A == "TGFB1|L")
tmp_cols <- c("source", "Ligand", "Receptor", "target")
data$freq <-1
upsel <- dplyr::slice_max(data, order_by = data$LRScore,
                        n = ifelse(dim(data)[1] > threshold, threshold, dim(data)[1]), with_ties = FALSE)
lowsel <- dplyr::slice_min(data, order_by = data$LRScore,
                        n = ifelse(dim(data)[1] > threshold, threshold, dim(data)[1]), with_ties = FALSE)
tmp = rbind(upsel,lowsel)
ggplot2::ggplot(tmp, aes(y = .data$freq, axis1 = .data$source,
                               axis2 = stats::reorder(.data$gene_A, -.data$LRScore),
                               axis3 = stats::reorder(.data$gene_B, -.data$LRScore),
                               axis4 = .data$target)) +
        ggalluvial::geom_alluvium(aes(fill = .data$LRScore, color = 'b'),
                                  width = 1 / 12,
                                  discern = FALSE) +
        ggalluvial::geom_stratum(width = 1 / 12) +
        ggplot2::geom_label(stat = ggalluvial::StatStratum,
                            ggplot2::aes(label = ggplot2::after_stat(.data$stratum)),
                            size = 4) +
        ggplot2::scale_x_discrete(limits = tmp_cols, expand = c(.05, .05)) +
        ggplot2::scale_fill_gradient2(low = colorBlindness::Blue2DarkOrange18Steps[4],
                                      mid = colorBlindness::Blue2DarkOrange18Steps[10],
                                      high = colorBlindness::Blue2DarkOrange18Steps[14], midpoint = 0) +
        ggplot2::scale_color_manual(values = c("black")) +
        ggplot2::theme(text = element_text(size = 8)) +
        ggplot2::theme_minimal()
#ggsave("dCM_sankey.pdf",width=20,height=10)

In [ ]:
sessionInfo()